In [3]:
import urllib
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 

Connect to the local sql express database.

In [4]:
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=DESKTOP-LSOCJD8\SQLEXPRESS;"
                                 "DATABASE=NCAA_Basketball;"
                                 "Trusted_Connection=yes")

engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
connection = engine.connect()
result = connection.execute("SELECT SYSTEM_USER AS me")
row = result.fetchone()
print(row['me'])

MicrosoftAccount\chipk215@msn.com


Get games for training.
This function reads all of the data into memory.

In [27]:
def get_games_for_training(season, db_connect):
    sql = """
    SELECT  game_id, season, scheduled_date, 
        home_team, market, opp_market, win, 
        points_game, opp_points_game,
        field_goals_pct, 
        offensive_rebounds, 
        free_throws_att,
        free_throws_pct,
        turnovers      
    FROM [NCAA_Basketball].[dbo].[d1_2015]
    WHERE (season = {}) 
     """.format(season)
    
    games_df = pd.read_sql_query(sql, connection)
    return games_df
    

Get the team information for the specified season and team name.
Revisit and only pull data prior to a game date (game date needs to be provided as a parameter)

In [54]:
def get_games_for_team(season, team, game_date, db_connect):
    if ("'" in team):
        pos = team.find("'")
        team = team[: pos]
        print(team)
        
    sql = """
    SELECT scheduled_date, home_team,market, opp_market, win, 
        points_game, opp_points_game,
        field_goals_pct, 
        offensive_rebounds, 
        free_throws_att,
        free_throws_pct,
        turnovers      
    FROM [NCAA_Basketball].[dbo].[d1_2015]
     WHERE (season = {}) AND (market like  N'{}%') AND scheduled_date < '{}'
     """.format(season, team, game_date.strftime("%Y-%m-%d"))
    
    team_df = pd.read_sql_query(sql, connection)
    return team_df
    

In [15]:
# Compute the win percentage for a team.
def compute_win_percentage(team_df):
    return team_df[team_df.win == True].count()['win']/team_df.shape[0]

In [16]:
def process_game_record(season, game_date, principal_team, opp_team):
    #print("Principal=",principal_team, " Opponent= ", opp_team)
    principal_df = get_games_for_team(season, principal_team, game_date, connection )
    opponent_df =  get_games_for_team(season, opp_team,game_date, connection )
    result = compute_feature_differences(principal_df, opponent_df)
    return result

In [56]:
def compute_feature_differences(principal_df, opponent_df):
    principal_summary = principal_df.describe()
    opponent_summary = opponent_df.describe()
    diff = principal_summary.loc['mean'] - opponent_summary.loc['mean']
    principal_vs_opponent = diff.to_frame(name="Difference").transpose()
    win_diff = compute_win_percentage(principal_df) -  compute_win_percentage(opponent_df)  
    principal_vs_opponent['win_pct'] = pd.Series(win_diff, index=principal_vs_opponent.index)
    return principal_vs_opponent

In [37]:
games_df = get_games_for_training(2015, connection)

games_df.info()
#games_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 14 columns):
game_id               5515 non-null object
season                5515 non-null int64
scheduled_date        5515 non-null datetime64[ns]
home_team             5515 non-null bool
market                5515 non-null object
opp_market            5515 non-null object
win                   5515 non-null bool
points_game           5515 non-null int64
opp_points_game       5515 non-null int64
field_goals_pct       5515 non-null float64
offensive_rebounds    5515 non-null int64
free_throws_att       5515 non-null int64
free_throws_pct       5515 non-null float64
turnovers             5515 non-null int64
dtypes: bool(2), datetime64[ns](1), float64(2), int64(6), object(3)
memory usage: 527.9+ KB


In [57]:
stats = games_df.apply(lambda x: process_game_record(x.season, x.scheduled_date, x.market, x.opp_market), axis=1)
game_stats_df = stats[0]
for row in range(1,stats.shape[0]):
    game_stats_df = pd.concat([game_stats_df,stats[row]], ignore_index=True)




KeyError: ('the label [mean] is not in the [index]', 'occurred at index 0')

In [39]:
game_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 8 columns):
points_game           5515 non-null float64
opp_points_game       5515 non-null float64
field_goals_pct       5515 non-null float64
offensive_rebounds    5515 non-null float64
free_throws_att       5515 non-null float64
free_throws_pct       5515 non-null float64
turnovers             5515 non-null float64
win_pct               5515 non-null float64
dtypes: float64(8)
memory usage: 344.8 KB


In [41]:
# this is the feature dataframe with some convenience columns added

game_stats_df['game_id'] = games_df['game_id']
game_stats_df['market'] = games_df['market']
game_stats_df['opp_market'] = games_df['opp_market']
game_stats_df['game_result'] = games_df['win'].astype(int)
game_stats_df['game_date'] = games_df['scheduled_date']
game_stats_df

,points_game,opp_points_game,field_goals_pct,offensive_rebounds,free_throws_att,free_throws_pct,turnovers,win_pct,game_id,market,opp_market,game_result,game_date
0,7.170455,17.136364,0.609091,-6.545455,-2.397727,6.072727,-0.931818,0.102273,e302d3cb-bbeb-47fa-9f96-0d85eb176c86,Fairfield,Yale,0,2015-11-13
1,-0.181818,-4.216783,4.993007,-1.534965,-10.314685,-9.868881,-0.104895,0.164336,6927bc52-8608-42ab-9d1c-504c48211edd,Iowa State,Colorado,1,2015-11-13
2,6.303030,1.878788,-1.936364,4.939394,1.651515,0.786364,-0.909091,0.045455,e08607b9-ff8f-4bfd-b62d-e49a2c31bb35,Eastern Michigan,Vermont,1,2015-11-13
3,4.452991,-12.427350,4.996581,1.589744,-4.692308,9.941026,-0.666667,0.504274,9b406ba4-1b74-4d22-b92e-3885ee7e6fe1,Wagner,St. John's,0,2015-11-13
4,7.025926,4.755556,2.335185,2.911111,1.251852,8.192222,0.381481,0.092593,083f7e42-8276-4f78-945e-cae3c0a9d401,South Dakota,Wright State,0,2015-11-13
5,0.666667,18.000000,0.383333,-2.333333,-9.666667,7.566667,4.166667,-0.500000,af588f0f-e314-4f49-b429-dd50d65de7fb,Northwestern State,Ole Miss,0,2015-11-13
6,3.400000,-2.190909,2.354545,0.609091,-3.190909,-7.514545,-4.127273,0.163636,669c3a74-a1af-4153-8e9b-2445dbd8e3d0,Pennsylvania,Robert Morris,1,2015-11-13
7,14.000000,1.285714,9.328571,-0.857143,2.785714,-0.742857,-0.928571,0.500000,2ab37176-3e33-483a-b3d6-5e2f28361dc2,Central Michigan,Jacksonville State,1,2015-11-13
8,6.616667,8.716667,9.565000,-7.283333,6.866667,0.765000,1.116667,0.050000,a4781f2d-9d18-4904-8cfb-8557b35ac13f,Sacred Heart,Quinnipiac,1,2015-11-13
9,4.463636,-7.463636,-2.023636,3.727273,1.190909,4.599091,-1.245455,0.354545,69518d00-8817-4bfc-8e39-d300584c0935,TCU,Southeastern Louisiana,1,2015-11-14


In [42]:
#game_stats_df.to_csv('D1_2015_Processed_Stats.csv')

In [44]:
game_record = games_df.loc[0]
game_record

game_id               e302d3cb-bbeb-47fa-9f96-0d85eb176c86
season                                                2015
scheduled_date                         2015-11-13 00:00:00
home_team                                            False
market                                           Fairfield
opp_market                                            Yale
win                                                  False
points_game                                             57
opp_points_game                                         70
field_goals_pct                                         40
offensive_rebounds                                       4
free_throws_att                                         13
free_throws_pct                                       76.9
turnovers                                               15
Name: 0, dtype: object

In [45]:
type(game_record.scheduled_date)
    

pandas._libs.tslibs.timestamps.Timestamp

In [47]:
temp_date = game_record.scheduled_date.date()
type(temp_date)

datetime.date

In [48]:
temp_date

datetime.date(2015, 11, 13)

In [49]:
date_string = f'{temp_date:%Y-%m-%d}'
date_string

'2015-11-13'

In [51]:
s2 = temp_date.strftime("%Y-%m-%d")
s2

'2015-11-13'